In [1]:
%matplotlib inline
%load_ext tensorboard

import matplotlib.pyplot as plt
import nengo
import numpy as np
import tensorflow as tf

import datetime, os
from itertools import islice
from IPython.display import clear_output
import pytry
import pickle
import random

import nengo_dl

C:\Users\mbartlett2\AppData\Local\Continuum\anaconda3\lib\site-packages\nengo_dl\converter.py:305: UserWarning: Layer '<class 'tensorflow.python.keras.layers.normalization_v2.BatchNormalization'>' already has a converter. Overwriting.
  "Layer '%s' already has a converter. Overwriting." % keras_layer


# Deep Learning Nengo Network

Tensorflow version 2.2.0 <br>
Nengo dl version 3.0.0

In [2]:
class FeedForwardTrain(pytry.Trial):
    def params(self):
        self.param('run_n', run_n=1)
        self.param('theta', theta=3)
        self.param('q', q=4)
        self.param('lr', lr=0.001)
        self.param('epoch', epoch=1000)
    
    def evaluate(self, param):       
        #LOAD/SET PARAMETERS
        run_n=param.run_n     
        theta=param.theta  
        q=param.q  
        learn_rate=param.lr #learning rate
        n_epoch=param.epoch #number of training epochs
        train_rat=0.7 #ratio of training to testing data
        
        print(run_n) #print run number
        np.random.seed() #set seed to random value
        
#################################################################
        #Set and create data directories
        data_dir="D:\\NEN002\\GridSearch_NDL_LMU\\"+str(learn_rate) #directory to save data to
        
        if run_n == 0:
            os.mkdir(data_dir)
            os.mkdir("D:\\NEN002\\GridSearch_NDL_LMU_logs\\"+str(learn_rate))
            
        logdir = os.path.join("D:\\NEN002\\GridSearch_NDL_LMU_logs\\"+str(learn_rate)+"\\"+str(run_n)) #directory to save logs to for tensorboard
        
        #load data
        highengdata = np.load('high_lmu'+str(q)+'_theta'+str(theta)+'.npy',allow_pickle=True) #np.load('higheng.npy', allow_pickle=True)
        lowengdata = np.load('low_lmu'+str(q)+'_theta'+str(theta)+'.npy',allow_pickle=True) #np.load('loweng.npy', allow_pickle=True)
##################################################################

        #high=list(highengdata[:])
        #low=list(lowengdata[:])
        
        high=list(highengdata) #[:,1])
        low=list(lowengdata) #[:,1])

        #shuffle clips
        np.random.shuffle(high)
        np.random.shuffle(low)

        #get training clips and convert to list of frames
        high_train = high[:(int(len(low)*train_rat))]
        high_train=list(np.vstack(high_train))
        low_train = low[:(int(len(low)*train_rat))]
        low_train=list(np.vstack(low_train))

        #shuffle frames
        np.random.shuffle(high_train)
        np.random.shuffle(low_train)

        #Make each training set same size (low is shortest)
        high_train = high_train[:(int(len(low_train)))]
        low_train = low_train[:(int(len(low_train)))]

        #Create 10% validation dataset
        high_val = high[(int(len(low)*0.9)):(int(len(low)*1))]
        low_val = low[(int(len(low)*0.9)):(int(len(low)*1))]
        
        #convert to list of frames
        high_val=list(np.vstack(high_val))
        low_val=list(np.vstack(low_val))

        #shuffle frames
        np.random.shuffle(high_val)
        np.random.shuffle(low_val)
        
        #create test sets
        #get 30% of clips for testing
        high_test = high[(int(len(low)*train_rat)):(int(len(low)*0.9))]
        low_test = low[(int(len(low)*train_rat)):(int(len(low)*0.9))]

        #shuffle frames
        np.random.shuffle(high_test)
        np.random.shuffle(low_test)

        #GENERATE INPUT MATRICES
        #Concatenate high and low sets together to create a single array for training, test and validation separately
        all_train = np.vstack(np.concatenate((high_train, low_train)))
        all_test = np.concatenate((high_test, low_test)) 
        all_val = np.concatenate((high_val, low_val))

        #create the target data for training
        target_train = np.zeros((all_train.shape[0],2))
        n_high = len(high_train)
        target_train[:n_high,0] = 1 #target for high = [1,0]
        target_train[n_high:,1] = 1 #target for low = [0,1]
        
        #and for validation
        target_val = np.zeros((all_val.shape[0],2))
        n_high = len(high_val)
        target_val[:n_high,0] = 1
        target_val[n_high:,1] = 1
        
        #SAVE TRAINING AND TESTING DATA
        pickle_filename = (data_dir+"/%s_training_data.pkl") % str(run_n) #(data_dir+"/%s_training_data"+str(test_param)+".pkl") % run_n
        with open(pickle_filename, 'wb') as file:
            pickle.dump(all_train, file)

        pickle_filename = (data_dir+"/%s_testing_data.pkl") % str(run_n) #(data_dir+"/%s_testing_data"+str(test_param)+".pkl") % run_n
        with open(pickle_filename, 'wb') as file:
            pickle.dump(all_test, file)

        #BUILD MODEL (input layer -> hidden layer -> output layer)
        N = 200
        seed = 1
        N_dims = len(all_train[0]) 

        model = nengo.Network(seed=seed)
        with model:    
            input = nengo.Node(np.zeros(N_dims))
            hidden1 = nengo.Ensemble(n_neurons=N, dimensions=N_dims, radius=np.sqrt(N_dims), 
                                     neuron_type=nengo.RectifiedLinear())
            nengo.Connection(input, hidden1, synapse=None)
            output = nengo.Node(None, size_in=2)
            nengo.Connection(hidden1, output, eval_points=all_train, function=target_train, 
                             scale_eval_points=False, synapse=None)

            p_output = nengo.Probe(output)
        
        #TRAINING DATA AND PARAMETERS
        minibatch_size = 1000       # this can be adjusted to speed up training
                                    # Note: with larger minibatch_size, you may need to reduce this
                                    #(if training is making things worse, then you need to reduce this!)
                                    # number of iterations through the training data to perform

        batches = int(np.ceil(len(all_train)/minibatch_size)) #number of training batches needed (length of data / minibatch size)
        val_batches = int(np.ceil(len(all_val)/minibatch_size)) #number of validation batches needed

        training_data_input = np.array(all_train, copy=True)
        order = np.arange(len(training_data_input))
        np.random.shuffle(order)
        training_data_input_rand = training_data_input[order]
        training_data_input_rand.resize(minibatch_size,batches,N_dims)
        training_data_input.resize(minibatch_size,batches,N_dims)

        training_data_output = np.array(target_train, copy=True)
        training_data_output_rand = training_data_output[order]
        training_data_output.resize(minibatch_size,batches,2)
        training_data_output_rand.resize(minibatch_size,batches,2)
        
        val_input = np.array(all_val, copy=True)
        order = np.arange(len(val_input))
        np.random.shuffle(order)
        val_input_rand = val_input[order]
        val_input_rand.resize(minibatch_size,val_batches,N_dims)
        val_input.resize(minibatch_size,val_batches,N_dims)
        
        val_output = np.array(target_val, copy=True)
        val_output_rand = val_output[order]
        val_output.resize(minibatch_size,val_batches,2)
        val_output_rand.resize(minibatch_size,val_batches,2)
        
        #set learning objective (defines loss)
        def objective(outputs, targets): 
            return tf.compat.v1.nn.softmax_cross_entropy_with_logits_v2(
                logits=outputs, labels=targets)  
        
        #TRAINING
        with nengo_dl.Simulator(
                model, minibatch_size=minibatch_size) as sim:
            sim.compile(optimizer=tf.optimizers.Adam(learn_rate),
                loss={p_output: objective})
            sim.fit(training_data_input_rand, training_data_output_rand, epochs=n_epoch,
                   callbacks=[tf.keras.callbacks.TensorBoard(log_dir=logdir)],
                   validation_data=(val_input_rand, val_output_rand))
            sim.save_params(data_dir+"/%s_trained.data" % str(run_n)) #(data_dir+"/%s"+str(test_param)+"_trained.data") % run_n #('./trained.data') 
            
        #Run model with training data, using trained weights
        input.output = nengo.processes.PresentInput(all_train, presentation_time=0.001)
        with nengo_dl.Simulator(model, minibatch_size=1) as sim:
            sim.load_params(data_dir+"/%s_trained.data" % str(run_n)) #(data_dir+"/%s"+str(test_param)+"_trained.data") % run_n
            sim.run(len(all_train)*0.001)
            
        #Save data
        pickle_filename = (data_dir+"/%s_train_target.pkl") % str(run_n) #(data_dir+"/%s_train_target"+str(test_param)+".pkl") % run_n
        with open(pickle_filename, 'wb') as file:
            pickle.dump(target_train, file)
        
        pickle_filename = (data_dir+"/%s_train_trange.pkl") % str(run_n) #(data_dir+"/%s_train_trange"+str(test_param)+".pkl") % run_n
        with open(pickle_filename, 'wb') as file:
            pickle.dump(sim.trange(), file)
            
        pickle_filename = (data_dir+"/%s_train_out.pkl") % str(run_n) #(data_dir+"/%s_train_out"+str(test_param)+".pkl") % run_n
        with open(pickle_filename, 'wb') as file:
            pickle.dump(sim.data[p_output][0], file)    
        
        test_out=[]
        target_test=[]
        for j in range(len(all_test)):
            if j<int(len(high_test)):
                target = np.zeros((all_test[j].shape[0],2))
                target[:,0] = 1
            else:
                target = np.zeros((all_test[j].shape[0],2))
                target[:,1] = 1
            input.output = nengo.processes.PresentInput(all_test[j], presentation_time=0.001)
            with nengo_dl.Simulator(model, minibatch_size=1) as sim:
                sim.load_params(data_dir+"/%s_trained.data" % str(run_n)) #(data_dir+"/%s"+str(test_param)+"_trained.data") % run_n
                sim.run(len(all_test[j])*0.001)
            test_out.append(sim.data[p_output][0])
            target_test.append(target)
            
        #Save data
        pickle_filename = (data_dir+"/%s_test_target.pkl") % str(run_n) #(data_dir+"/%s_test_target"+str(test_param)+".pkl") % run_n
        with open(pickle_filename, 'wb') as file:
            pickle.dump(target_test, file)
        
        pickle_filename = (data_dir+"/%s_test_trange.pkl") % str(run_n) #(data_dir+"/%s_test_trange"+str(test_param)+".pkl") % run_n
        with open(pickle_filename, 'wb') as file:
            pickle.dump(sim.trange(), file)
            
        pickle_filename = (data_dir+"/%s_test_out.pkl") % str(run_n) #(data_dir+"/%s_test_out"+str(test_param)+".pkl") % run_n
        with open(pickle_filename, 'wb') as file:
            pickle.dump(test_out, file)


In [ ]:
#for run in range(3,9):
#for lr in (0.001, 0.0001, 0.00001, 0.000001):
FeedForwardTrain().run(run_n=9, lr=0.1) 

running FeedForwardTrain#20200720-105345-0b1a8c39
9
|                     Building network (0%)                    | ETA:  --:--:--

C:\Users\mbartlett2\AppData\Local\Continuum\anaconda3\lib\site-packages\nengo_dl\simulator.py:460: UserWarning: No GPU support detected. It is recommended that you install tensorflow-gpu (`pip install tensorflow-gpu`).
  "No GPU support detected. It is recommended that you "


Build finished in 0:00:02                                                      
Optimization finished in 0:00:00                                               
Train on 1000 samples, validate on 1000 samples


Epoch 1/1000
1000/1000 [==============================] - 2s 2ms/sample - loss: 0.8014 - probe_loss: 0.8014 - val_loss: -1587.0345 - val_probe_loss: -1587.0345
Epoch 2/1000
1000/1000 [==============================] - 1s 937us/sample - loss: -1583.2781 - probe_loss: -1583.2781 - val_loss: -3176.9194 - val_probe_loss: -3176.9194
Epoch 3/1000
1000/1000 [==============================] - 1s 833us/sample - loss: -3205.2412 - probe_loss: -3205.2412 - val_loss: -4774.3823 - val_probe_loss: -4774.3823
Epoch 4/1000
1000/1000 [==============================] - 1s 826us/sample - loss: -4881.9946 - probe_loss: -4881.9946 - val_loss: -6382.0430 - val_probe_loss: -6382.0430
Epoch 5/1000
1000/1000 [==============================] - 1s 890us/sample - loss: -6629.8496 - probe_loss: -6629.8496 - val_loss: -8002.2847 - val_probe_loss: -8002.2847
Epoch 6/1000
1000/1000 [==============================] - 1s 891us/sample - loss: -8468.8193 - probe_loss: -8468.8193 - val_loss: -9635.2725 - val_probe_loss: -

1000/1000 [==============================] - 1s 946us/sample - loss: -1269634.7500 - probe_loss: -1269634.7500 - val_loss: -739246.6875 - val_probe_loss: -739246.6875
Epoch 48/1000
1000/1000 [==============================] - 1s 951us/sample - loss: -1380227.0000 - probe_loss: -1380227.0000 - val_loss: -800872.6875 - val_probe_loss: -800872.6875
Epoch 49/1000
1000/1000 [==============================] - 1s 1ms/sample - loss: -1498366.6250 - probe_loss: -1498366.6250 - val_loss: -866580.7500 - val_probe_loss: -866580.7500
Epoch 50/1000
1000/1000 [==============================] - 1s 1ms/sample - loss: -1624418.6250 - probe_loss: -1624418.6250 - val_loss: -936572.6250 - val_probe_loss: -936572.6250
Epoch 51/1000
1000/1000 [==============================] - 1s 927us/sample - loss: -1758762.7500 - probe_loss: -1758762.7500 - val_loss: -1011055.0000 - val_probe_loss: -1011055.0000
Epoch 52/1000
1000/1000 [==============================] - 1s 978us/sample - loss: -1901787.0000 - probe_loss: 

Epoch 92/1000
1000/1000 [==============================] - 1s 831us/sample - loss: -20416726.0000 - probe_loss: -20416726.0000 - val_loss: -11211882.0000 - val_probe_loss: -11211882.0000
Epoch 93/1000
1000/1000 [==============================] - 1s 836us/sample - loss: -21370112.0000 - probe_loss: -21370112.0000 - val_loss: -11734540.0000 - val_probe_loss: -11734540.0000
Epoch 94/1000
1000/1000 [==============================] - 1s 781us/sample - loss: -22357272.0000 - probe_loss: -22357272.0000 - val_loss: -12276047.0000 - val_probe_loss: -12276047.0000
Epoch 95/1000
1000/1000 [==============================] - 1s 827us/sample - loss: -23379010.0000 - probe_loss: -23379010.0000 - val_loss: -12836862.0000 - val_probe_loss: -12836862.0000
Epoch 96/1000
1000/1000 [==============================] - 1s 890us/sample - loss: -24436124.0000 - probe_loss: -24436124.0000 - val_loss: -13417470.0000 - val_probe_loss: -13417470.0000
Epoch 97/1000
1000/1000 [==============================] - 1s 924

Epoch 136/1000
1000/1000 [==============================] - 1s 911us/sample - loss: -105394248.0000 - probe_loss: -105394248.0000 - val_loss: -58537148.0000 - val_probe_loss: -58537148.0000
Epoch 137/1000
1000/1000 [==============================] - 1s 1ms/sample - loss: -108636576.0000 - probe_loss: -108636576.0000 - val_loss: -60363300.0000 - val_probe_loss: -60363300.0000
Epoch 138/1000
1000/1000 [==============================] - 1s 1ms/sample - loss: -111950984.0000 - probe_loss: -111950984.0000 - val_loss: -62231292.0000 - val_probe_loss: -62231292.0000
Epoch 139/1000
1000/1000 [==============================] - 1s 988us/sample - loss: -115338384.0000 - probe_loss: -115338384.0000 - val_loss: -64141684.0000 - val_probe_loss: -64141684.0000
Epoch 140/1000
1000/1000 [==============================] - 1s 862us/sample - loss: -118799744.0000 - probe_loss: -118799744.0000 - val_loss: -66095012.0000 - val_probe_loss: -66095012.0000
Epoch 141/1000
1000/1000 [============================

Epoch 179/1000
1000/1000 [==============================] - 1s 793us/sample - loss: -321117536.0000 - probe_loss: -321117536.0000 - val_loss: -182132976.0000 - val_probe_loss: -182132976.0000
Epoch 180/1000
1000/1000 [==============================] - 1s 785us/sample - loss: -328269760.0000 - probe_loss: -328269760.0000 - val_loss: -186284912.0000 - val_probe_loss: -186284912.0000
Epoch 181/1000
1000/1000 [==============================] - 1s 680us/sample - loss: -335531520.0000 - probe_loss: -335531520.0000 - val_loss: -190502816.0000 - val_probe_loss: -190502816.0000
Epoch 182/1000
1000/1000 [==============================] - 1s 794us/sample - loss: -342903648.0000 - probe_loss: -342903648.0000 - val_loss: -194787360.0000 - val_probe_loss: -194787360.0000
Epoch 183/1000
1000/1000 [==============================] - 1s 742us/sample - loss: -350387168.0000 - probe_loss: -350387168.0000 - val_loss: -199138640.0000 - val_probe_loss: -199138640.0000
Epoch 184/1000
1000/1000 [==============

Epoch 222/1000
1000/1000 [==============================] - 1s 824us/sample - loss: -737300288.0000 - probe_loss: -737300288.0000 - val_loss: -426125120.0000 - val_probe_loss: -426125120.0000
Epoch 223/1000
1000/1000 [==============================] - 1s 893us/sample - loss: -749868352.0000 - probe_loss: -749868352.0000 - val_loss: -433540000.0000 - val_probe_loss: -433540000.0000
Epoch 224/1000
1000/1000 [==============================] - 1s 766us/sample - loss: -762578752.0000 - probe_loss: -762578752.0000 - val_loss: -441040960.0000 - val_probe_loss: -441040960.0000
Epoch 225/1000
1000/1000 [==============================] - 1s 830us/sample - loss: -775432448.0000 - probe_loss: -775432448.0000 - val_loss: -448628000.0000 - val_probe_loss: -448628000.0000
Epoch 226/1000
1000/1000 [==============================] - 1s 770us/sample - loss: -788430272.0000 - probe_loss: -788430272.0000 - val_loss: -456302080.0000 - val_probe_loss: -456302080.0000
Epoch 227/1000
1000/1000 [==============

1000/1000 [==============================] - 1s 972us/sample - loss: -1396331136.0000 - probe_loss: -1396331136.0000 - val_loss: -816562816.0000 - val_probe_loss: -816562816.0000
Epoch 265/1000
1000/1000 [==============================] - 1s 981us/sample - loss: -1415515264.0000 - probe_loss: -1415515264.0000 - val_loss: -827961472.0000 - val_probe_loss: -827961472.0000
Epoch 266/1000
1000/1000 [==============================] - 1s 728us/sample - loss: -1434871552.0000 - probe_loss: -1434871552.0000 - val_loss: -839463936.0000 - val_probe_loss: -839463936.0000
Epoch 267/1000
1000/1000 [==============================] - 1s 668us/sample - loss: -1454401024.0000 - probe_loss: -1454401024.0000 - val_loss: -851071040.0000 - val_probe_loss: -851071040.0000
Epoch 268/1000
1000/1000 [==============================] - 1s 1ms/sample - loss: -1474105088.0000 - probe_loss: -1474105088.0000 - val_loss: -862782848.0000 - val_probe_loss: -862782848.0000
Epoch 269/1000
1000/1000 [=====================

To view logs in tensorboard use following command in command prompt: <br>
    tensorboard --logdir={logs_base_dir} --host=localhost <br>
<br>
Note: change {logs_base_dir} to the name of the folder where logs are being saved. <br>
Note: make sure you have navigated to the directory containing this script in the command prompt before running the above command.